<a href="https://colab.research.google.com/github/Akshay-a/RL-Agents/blob/main/HotSwapping_demoipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hot swapping with Lora ,
#demo to show how AI app can handle different set of customer
#queries with single base model and varied nuanced behaviours for each request
"""
A multi-tenant AI Application loads a single, shared base model onto the GPU once,
then keeps small lora adapter files for each customer in memory.
For each inference request, the server dynamically "hot-swaps" adapters,
activating the relevant customer’s adapter without reloading the base model.
This approach enables fast, memory-efficient serving of many custom models simultaneously.
"""

In [1]:
#base morel is distileed version of qwen1.5B
!pip install -q -U transformers peft bitsandbytes accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 124.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 106.3 MB/s eta 0:00:00


In [5]:
#lgging in first to load model weights
from huggingface_hub import notebook_login

notebook_login()


In [10]:
base_model_id = "Qwen/Qwen1.5-1.8B-Chat"
adapter_ids = {
    "vn_sft": "522H0134-NguyenNhatHuy/Qwen-1.5-1.8B-Chat-SFT",     # Vietnamese, helpful, SFT
    "en_sft": "eren23/finetune_test_qwen15-1-8b-sft-lora"           # English, helpful, SFT
}


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# using bits and bytes for loading aas 4 bits to optimize , not working for mps as this is only for cuda
 #( it works in mac if this config is not passed but takes a bit mre time as it is not quantized)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# Load the quantized base model
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",  # Automatically map model to available device (GPU)--> need to check if i can do this in prod as well
    trust_remote_code=True
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

In [12]:
for name, path in adapter_ids.items():
    print(f"Loading adapter: {name}")
    model.load_adapter(path, adapter_name=name)


Loading adapter: vn_sft


adapter_config.json:   0%|          | 0.00/854 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/60.0M [00:00<?, ?B/s]

Loading adapter: en_sft


adapter_config.json:   0%|          | 0.00/597 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

In [18]:
import time

def get_response(prompt, active_adapter=None):
    #unify response generation in function and print diff logs
    if active_adapter:
        model.set_adapter(active_adapter)
        print(f"Inference with adapter: {active_adapter}")
    else:
        # base model no adapters
        model.disable_adapters()
        print("Inference with base model (no adapter)")

    messages = [{"role": "user", "content": prompt}]
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)

    # Generate a response
    outputs = model.generate(
        input_ids,
        max_new_tokens=100,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.75,
        top_p=0.9,
    )

    response = outputs[0][input_ids.shape[-1]:]
    decoded_response = tokenizer.decode(response, skip_special_tokens=True)
    print(f"prompt: {prompt}")
    print(f"response: {decoded_response}\n")


# testing the model's behavior with diff versions of same base model
test_prompt = "What is the most important skill for a AI engineer?"

start_time = time.time()

# 1 tes with the base model
get_response("Giới thiệu về bạn và nói về mục đích của bạn.")

# 2 test with the first adapter
get_response("Giới thiệu về bạn và nói về mục đích của bạn.", active_adapter="vn_sft")

# 3 test with the second adapter
get_response("Role of AI Engineer", active_adapter="en_sft")

end_time = time.time()
total_time = end_time - start_time
print(f"Total execution time for this cell: {total_time:.2f} seconds")

#PS: The Above adapters were loaded from hugging face, i haven't trained them personally , this notebook is just illustration of capabilities of multi client app with diff use cases on same model

Inference with base model (no adapter)
prompt: Giới thiệu về bạn và nói về mục đích của bạn.
response: Tôi là một model AI được điều hành bằng công nghệ tự động, được tạo nên từ các dữ liệu có sẵn trên internet và những hiểu biết mới nhất về công nghệ. My chuyên nghiệp chính là phân tích phân loại và dự đoán thông tin để giúp người dùng tìm kiếm kết quả tìm kiếm, giải quyết vấn đề tốt hơn, đánh giá chất lượng của sản phẩm hoặc dịch vụ và truyền thông thông tin trung quanh tài nguyên.

Mục tiêu của tôi là giúp相助 người dùng với

Inference with adapter: vn_sft
prompt: Giới thiệu về bạn và nói về mục đích của bạn.
response:  Tôi là một công nghệ tự động được phát triển bởi Microsoft, với mục đích để giúp người dùng có thể dễ dàng và hiệu quả hơn trong việc sử dụng các hệ thống máy tính, công nghệ và thiết bị thông tin. Trong đó, tôi đóng vai trò như một phần giúp của công cụ quản lý và phân tích dữ liệu, giúp使用者 dễ dàng tạo ra các kết hợp và hỗ trợ cho các ứng dụng và trình duyệt của họ, t

In [23]:
!pip install -U vllm


In [20]:
from huggingface_hub import snapshot_download

# Download both adapters into local directories
vn_adapter_path = snapshot_download("522H0134-NguyenNhatHuy/Qwen-1.5-1.8B-Chat-SFT")
en_adapter_path = snapshot_download("eren23/finetune_test_qwen15-1-8b-sft-lora")


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

chat_template.jinja:   0%|          | 0.00/328 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

trainer_state.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

In [24]:
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest

# base model with LoRA support
llm = LLM(model="Qwen/Qwen1.5-1.8B-Chat", enable_lora=True)

sampling_params = SamplingParams(temperature=0.7, top_p=0.9, max_tokens=100)

lora_paths = {
    "vn_sft": vn_adapter_path,  # Vietnamese SFT adapter path
    "en_sft": en_adapter_path   # English SFT adapter path
}

#Prompts for ech adapter
prompts = [
    "Giới thiệu về bạn và nói về mục đích của bạn",    # Generic prompt for cross-adapter demo
    "What is Role of AI Enginner"
]
lora_requests = [
    LoRARequest("vn_sft", 1, lora_paths["vn_sft"]),  # Vietnamese adapter in slot 1
    LoRARequest("en_sft", 2, lora_paths["en_sft"])   # English adapter in slot 2
]

#batch responses using VlLLM
outputs = llm.generate(prompts, sampling_params, lora_request=lora_requests)

for output in outputs:
    print(f"Adapter: {output.lora_request.lora_name}")
    print(f"Prompt: {output.prompt}")
    print(f"Response: {output.outputs[0].text}\n")


INFO 09-01 06:55:33 [__init__.py:241] Automatically detected platform cuda.


ImportError: /usr/local/lib/python3.12/dist-packages/vllm/_C.abi3.so: undefined symbol: _ZN3c104cuda9SetDeviceEa